# Global Climatology


Advect particles with climatological monthly mean flow at selected depth level
(e.g. `k=10` for 95 m) from a global ocean state estimate ([ECCO v4 r2](https://eccov4.readthedocs.io/en/latest/) ; see also <https://ecco-group.org>)
which is here repeated for `ny` years. For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[2](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

[![simulated particle movie (5m)](https://user-images.githubusercontent.com/20276764/84766999-b801ad80-af9f-11ea-922a-610ad8a257dc.png)](https://youtu.be/W5DNqJG9jt0)

## 1. Get Software & Iput Files

- packages + helper functions
- grid and velocity files

In [1]:
using IndividualDisplacements, MeshArrays, OrdinaryDiffEq
using Statistics, DataFrames, MITgcmTools, OceanStateEstimation

include(joinpath(dirname(pathof(IndividualDisplacements)),"../examples/helper_functions.jl"))
get_llc90_grid_if_needed(); get_ecco_velocity_if_needed();

## 2. Set Up Parameters & Inputs

- select vertical level & duration in years
- read grid variables & velocities
- normalize velocities

In [2]:
𝑃=setup_global_ocean(k=1,ny=2);

keys(𝑃)

(:u0, :u1, :v0, :v1, :𝑇, :🔄, :pth, :XC, :YC, :iDXC, :iDYC, :k, :msk, :aS, :RelocFunctions, :aW, :aN, :aE, :frac, :Γ)

## 3. Main Computation Loop

### 3.1 Initialize Individuals & Solution

- initial particle positions randomly over Global Ocean

In [3]:
xy=init_global_randn(1000,𝑃); id=collect(1:size(xy,2))
𝐼 = Individuals{Float64}(📌=xy[:,:], 🆔=id, 🔧=postprocess_lonlat, 🚄 = dxy_dt!, 𝑃=𝑃)

fieldnames(typeof(𝐼))

(:📌, :🔴, :🆔, :🚄, :∫, :🔧, :𝑃, :𝐷, :𝑀)

- initial integration from time 0 to 0.5 month

In [4]:
𝑇=(0.0,𝐼.𝑃.𝑇[2])
∫!(𝐼,𝑇)

3×1000 Array{Float64,2}:
  14.9542  178.469   97.1106  39.731   …  78.0433  185.775   140.93   
 150.623    53.7219  49.9654  29.2653     27.3099   77.7268    9.80923
   2.0       5.0      5.0      4.0         4.0       4.0       5.0    

### 3.2 Iteration function example

- `𝐼.𝑃.🔄(𝐼.𝑃,t_ϵ)` resets the velocity input streams to bracket t_ϵ=𝐼.𝑃.𝑇[2]+eps(𝐼.𝑃.𝑇[2])
- `reset_lonlat!(𝐼)` randomly selects a fraction (defined in `setup_global_ocean()`) of the particles and resets their positions before each integration period. This can maintain homogeneous coverage of the Global Ocean by particles.
- `∫!(𝐼,𝑇)` then solves for the individual trajectories over one month, after updating velocity fields (𝐼.u0 etc) if needed, and adds diagnostics to the DataFrame used to record / trace variables along the trajectory (𝐼.tr).

In [5]:
function step!(𝐼::Individuals)
    t_ϵ=𝐼.𝑃.𝑇[2]+eps(𝐼.𝑃.𝑇[2])
    𝐼.𝑃.🔄(𝐼.𝑃,t_ϵ)
    reset_lonlat!(𝐼)
    𝑇=Tuple(𝐼.𝑃.𝑇)
    ∫!(𝐼,𝑇)
end

step! (generic function with 1 method)

## 3.3 Iterate For `ny*12` Months

In [6]:
[step!(𝐼) for y=1:2, m=1:12]

2×12 Array{Array{Float64,2},2}:
 [14.0775 178.61 … 184.26 141.43; 150.356 61.0985 … 84.1326 10.8332; 2.0 5.0 … 4.0 5.0]     …  [82.1467 32.2991 … 13.0576 61.2429; 148.709 177.348 … 87.4093 59.7777; 1.0 2.0 … 5.0 5.0]
 [13.5483 172.935 … 184.094 141.757; 150.087 65.0049 … 89.5638 11.8183; 2.0 5.0 … 4.0 5.0]     [80.3167 25.5409 … 23.9533 62.1413; 146.958 175.819 … 85.3732 63.7219; 1.0 2.0 … 5.0 5.0]

## 3.4 Compute summary statistics

See [DataFrames.jl](https://juliadata.github.io/DataFrames.jl/latest/) documentation for detail and additinal functionalities.

In [7]:
gdf = groupby(𝐼.🔴, :ID)
sgdf= combine(gdf,nrow,:lat => mean)
sgdf[rand(1:size(sgdf,1),4),:]

,ID,nrow,lat_mean
,Float64,Int64,Float64
1,1226.0,62,-30.1888
2,649.0,761,24.3105
3,1091.0,465,76.5842
4,908.0,761,-47.0023


## 4. Plot trajectories / individual positions

```
using Plots
p=plot(;xlims=(-180,180),ylims=(-90,90),legend=:none)
p!(x,y)=scatter!(p,x,y,markersize=1.1,markerstrokewidth=0)
[p!(gdf[i].lon,gdf[i].lat) for i in rand(collect(1:length(gdf)),10)]
display(p)
```

Or select a background map (e.g. `lon`, `lat`, and `DL=log10(bottom depth)`)
and a recipe to superimpose initial and final locations. Try:

```
include(joinpath(dirname(pathof(IndividualDisplacements)),"../examples/recipes_plots.jl"))
plot_end_points(𝐼,𝐼.𝑃.)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*